# Google BigQuery 向量搜索

> [Google Cloud BigQuery 向量搜索](https://cloud.google.com/bigquery/docs/vector-search-intro) 可让您使用 GoogleSQL 进行语义搜索，使用向量索引获得快速的近似结果，或使用暴力搜索获得精确结果。

本教程将介绍如何在 LangChain 中处理端到端数据和嵌入管理系统，并通过 `BigQueryVectorStore` 类在 BigQuery 中提供可扩展的语义搜索。此类是能够为 Google Cloud 提供统一数据存储和灵活向量搜索的 2 个类的一部分：

- **BigQuery 向量搜索**: 使用 `BigQueryVectorStore` 类，非常适合无需基础设施设置即可进行快速原型设计和批量检索。
- **Feature Store Online Store**: 使用 `VertexFSVectorStore` 类，通过手动或计划数据同步实现低延迟检索。非常适合生产就绪的用户面向的 GenAI 应用程序。

![Diagram BQ-VertexFS]( QgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAp1FEBG3lP3WwBwQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQh0FgEi5o93QQAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAACEEAE3G84qQAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAGQQUQ5lB/gGgkAAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACBCyKADJmjhXhCAAAQgAAEIQAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACfRQg4t9/J4kBBCAAAQhAAAIQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACBCyKADJmjhXhCAAAQgAAEIQAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACDQQi5vbv/5K7xSghAAAIQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCCQYARCM7vU7AEAAAhCAAAQgAAEIqaA0I/tXJ1hXEQAAEIQAACEIAABCAAAQhAAAIQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAELoKACY+P3b4kAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACBCyKADJn4M0EAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhDooAQ2M7vPXBAAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACBCyKADJmjhXhCAAAQgAAEIQAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACfRAg5t9/J4kBBCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACLJCAkmgUEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhAAAEWjIAiZu9PntwQAIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhAAAEWRIMgZa6tB6wIAAIMAAAEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCCQXAQIcu593hQkAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCHRRgJCzK//jDQkAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQyC4C5+b4VncMQAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIACBWRAgytlj7YIAAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACCEAAAsYkIMjYyQIAAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCCQcgIEOWd83BQUAAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAARyKICDnJnu0hIAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEECgMAIECDLO+a+rAAIIIIAAAggggAACCCCQTQQQ5fzfAwABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABCDQYgQ5a3zXDlwQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABCCRAgIy/bJ8QQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQCCLAEHM8XvLBAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCJQ/AYKcpe+YUCMEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEMggQJAzAw6zEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEECg9AkQ5Cx9x4QaIYAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIBABgGCnBlwmIUAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAqVPgCBn6Tsm1AgBBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBDIIkCQM4GCH1hFAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAIEyJUj4XvrXbCYCCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACfgQIMj9R7cDAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACfQoQ5G8m7w0AAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQCCIBAhylYhW1RAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEAgQoAU5Cyx2DAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACfQQIMjfTd2nDAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACCCCAAAIIIIAAAggggAACXQkQ5G8m7w0AAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQCCIBAhylYhW1RAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEECgECFClhG854cAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABCCSB5Ajkf//xjQIAAQQQQAABBBBAAAEEEEAAAQQQQAABBBBAAAEEEEAAAQQQQAABBMoDAf832r+87kIAAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAQgAAEIQAACEIAABCAAAQhAAAIQgAAEIAABCEAAAhCAAAS6DAg52/f2rAAACAAAAAAAAAAAAAElFTkU5//fA/gAAAAAAABJRU5ErkJggg==)

## 入门

### 安装库

In [ ]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai "langchain-google-community[featurestore]"

为了在此 Jupyter运行时中使用新安装的软件包，您必须重启运行时。您可以通过运行下面的单元格来完成此操作，这将重启当前的内核。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 开始之前

#### 设置您的项目 ID

如果您不知道您的项目 ID，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 设置区域

您还可以更改 BigQuery 使用的 `REGION` 变量。 了解更多关于 [BigQuery 区域](https://cloud.google.com/bigquery/docs/locations#supported_locations) 的信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### 设置数据集和表名

它们将是您的 BigQuery 向量商店。

In [ ]:
DATASET = "my_langchain_dataset"  # @param {type: "string"}
TABLE = "doc_and_vectors"  # @param {type: "string"}

### 验证你的笔记本环境

- 如果你正在使用 **Colab** 运行此笔记本，请取消注释下面的单元格并继续。
- 如果你正在使用 **Vertex AI Workbench**，请查看此处的设置说明：[https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
# from google.colab import auth as google_auth

# google_auth.authenticate_user()

## Demo: BigQueryVectorStore

### 创建 embedding 类实例

您可能需要在项目中启用 Vertex AI API，方法是运行
`gcloud services enable aiplatform.googleapis.com --project {PROJECT_ID}`
（将 `{PROJECT_ID}` 替换为您的项目名称）。

您可以使用任何 [LangChain 文本嵌入模型](/docs/integrations/text_embedding/)。

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

### 初始化 BigQueryVectorStore

如果 BigQuery 数据集和表不存在，将会被自动创建。更多可选参数请参见此处的类定义：[here](https://github.com/langchain-ai/langchain-google/blob/main/libs/community/langchain_google_community/bq_storage_vectorstores/bigquery.py#L26)。

In [ ]:
from langchain_google_community import BigQueryVectorStore

store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
)

### 添加文本

In [ ]:
all_texts = ["Apples and oranges", "Cars and airplanes", "Pineapple", "Train", "Banana"]
metadatas = [{"len": len(t)} for t in all_texts]

store.add_texts(all_texts, metadatas=metadatas)

### 搜索文档

In [ ]:
query = "I'd like a fruit."
docs = store.similarity_search(query)
print(docs)

### 按向量搜索文档

In [ ]:
query_vector = embedding.embed_query(query)
docs = store.similarity_search_by_vector(query_vector, k=2)
print(docs)

### 使用元数据过滤器搜索文档
向量存储支持两种在执行文档搜索时应用元数据字段过滤器的方法：

-   基于字典的过滤器
    -   您可以传递一个字典（dict），其中键代表元数据字段，值指定过滤条件。此方法在键和相应值之间应用相等性过滤器。当提供多个键值对时，它们将通过逻辑 AND 操作组合。
-   基于 SQL 的过滤器
    -   或者，您可以提供一个表示 SQL WHERE 子句的字符串来定义更复杂的过滤条件。这提供了更大的灵活性，支持诸如比较运算符和逻辑运算符之类的 SQL 表达式。了解更多关于 [BigQuery 运算符](https://cloud.google.com/bigquery/docs/reference/standard-sql/operators)的信息。

In [ ]:
# Dictionary-based Filters
# This should only return "Banana" document.
docs = store.similarity_search_by_vector(query_vector, filter={"len": 6})
print(docs)

In [ ]:
# SQL-based Filters
# This should return "Banana", "Apples and oranges" and "Cars and airplanes" documents.
docs = store.similarity_search_by_vector(query_vector, filter="len = 6 AND len > 17")
print(docs)

### 批量搜索
BigQueryVectorStore 提供了 `batch_search` 方法，用于可扩展的向量相似性搜索。

In [ ]:
results = store.batch_search(
    embeddings=None,  # can pass embeddings or
    queries=["search_query", "search_query"],  # can pass queries
)

### 添加带有嵌入的文本

您也可以使用 `add_texts_with_embeddings` 方法添加自己的嵌入。
这对于可能需要自定义预处理才能生成嵌入的多模态数据特别有用。

In [ ]:
items = ["some text"]
embs = embedding.embed(items)

ids = store.add_texts_with_embeddings(
    texts=["some text"], embs=embs, metadatas=[{"len": 1}]
)

### 使用 Feature Store 实现低延迟服务
您只需使用 `.to_vertex_fs_vector_store()` 方法即可获取 VertexFSVectorStore 对象，该对象为在线用例提供低延迟服务。所有强制性参数将自动从现有的 BigQueryVectorStore 类传输。有关您可以使用所有其他参数的信息，请参阅[类定义](https://github.com/langchain-ai/langchain-google/blob/main/libs/community/langchain_google_community/bq_storage_vectorstores/featurestore.py#L33)。

使用 `.to_bq_vector_store()` 方法可以轻松地移回 BigQueryVectorStore。

In [ ]:
store.to_vertex_fs_vector_store()  # pass optional VertexFSVectorStore parameters as arguments